In [26]:
train_path1 = 'train_hand_fin.txt'
train_path2 = 'skweak_labelled_20_all.txt'
dev_path = 'dev_hand_fin.txt'
test_path = 'test_hand_fin.txt'

model_checkpoint = "bert-base-german-cased"

batch_size = 16
learning_rate = 5e-5
epoch = 3
weight_decay = 0.3

save_folder_name = 'bert_all'
save_model_name = 'bert_alldata_fin'

In [2]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 113.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
import re
import pandas as pd
from datasets import Dataset

Load dataset

In [4]:
def load_data_weak(path):
  file = open(path, 'r')
  lines = file.readlines()

  tokens = []
  ner_tags = []

  temp_toks = []
  temp_tags = []

  for i in lines:
    if i == ' \t \n':
      tokens.append(temp_toks)
      ner_tags.append(temp_tags)

      temp_toks = []
      temp_tags = []

    else:
      temp = re.split('\t',i)
      temp_toks.append(temp[0])
      temp_tags.append(re.sub('\n','',temp[-1]))
  
  data = pd.DataFrame({'tokens':tokens,
                       'ner_tags':ner_tags})
  return data


In [5]:
def load_data(path):
  file = open(path, 'r')
  lines = file.readlines()

  tokens = []
  ner_tags = []

  temp_toks = []
  temp_tags = []

  for i in lines:
    if i == '\n':
      tokens.append(temp_toks)
      ner_tags.append(temp_tags)

      temp_toks = []
      temp_tags = []

    else:
      temp = re.split('\t',i)
      temp_toks.append(temp[0])
      temp_tags.append(re.sub('\n','',temp[-1]))
  
  data = pd.DataFrame({'tokens':tokens,
                       'ner_tags':ner_tags})
  return data


In [7]:
train1 = load_data(train_path1)
train2 = load_data_weak(train_path2)
traindata = pd.concat([train1,train2])
traindata = traindata.reset_index(drop = True)

test = load_data(test_path)
dev = load_data(dev_path)

traindata.head()

,tokens,ner_tags
0,[ABKÜRZUNGSVERZEICHNIS],[O]
1,"[Empfehlung, Nr., 10, Ein, Überblick, der, Akt...","[O, O, O, O, O, O, O, O, O, O, B-ORG, I-ORG, O..."
2,"[Betreutes, Konto, Ist, ein, von, der, Schuldn...","[O, O, O, O, O, O, B-ORG, I-ORG, I-ORG, I-ORG,..."
3,"[Insgesamt, verfügte, die, Rettungsleitstelle,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[Empfehlungen, ,, die, in, einem, früheren, Be...","[O, O, O, O, O, O, O, O, B-ORG, I-ORG, O, O, O..."


In [11]:
label2id = {'O': 0,'B-ORG':1,'I-ORG':2}
id2label = {0 :'O', 1:'B-ORG', 2:'I-ORG'}
label_list = ['O','B-ORG','I-ORG']

In [12]:
from transformers import (
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer
)

In [13]:
config = AutoConfig.from_pretrained(
    model_checkpoint,
    num_labels=len(label_list),
    id2label = id2label,
    label2id = label2id
)


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    config=config
)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/485k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-b

In [15]:
def align_labels(data, label_encoding_dict):
    tokenized_inputs = tokenizer(data["tokens"],
                        max_length = 128, padding = 'max_length',
                        truncation=True, is_split_into_words=True)

    label_id_temp = {}
    for i, label in enumerate(data['ner_tags']):
        label_id_temp.update({i:label})

    word_ids = tokenized_inputs.word_ids(batch_index=0)

    labels = []
    for w_id in word_ids:
        if w_id == None:
            labels.append(-100)
        else:
            tag = label_id_temp[w_id]
            labels.append(label2id[tag])

    tokenized_inputs["labels"] = labels

    return tokenized_inputs

def tokenized_for_bert(df):
  for_bert = df.copy()
  for_bert['input_ids'] = ""
  for_bert['token_type_ids'] = ""
  for_bert['attention_mask']= ""
  for_bert['labels'] = ""

  for index, row in df.iterrows():
      inputs = align_labels(row,label2id)
      for_bert.at[index,'input_ids'] = inputs['input_ids']
      for_bert.at[index,'token_type_ids'] = inputs['token_type_ids']
      for_bert.at[index,'attention_mask'] = inputs['attention_mask']
      for_bert.at[index,'labels'] = inputs['labels']
  for_bert = for_bert.drop('tokens', axis = 1)
  for_bert = for_bert.drop('ner_tags', axis = 1)
  dataset_for_bert = Dataset.from_pandas(for_bert)
  print(dataset_for_bert)
  return dataset_for_bert

In [16]:
train_dataset = tokenized_for_bert(traindata)
dev_dataset = tokenized_for_bert(dev)
test_dataset = tokenized_for_bert(test)

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 13856
})
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1227
})
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1226
})


In [17]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 KB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16179 sha256=31a22cdcc71010cd9d10c8f78e21413980a483620cd5b4ae60b02c1d4f1d57c7
  Stored in directory: /root/.cache/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


In [18]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch
from seqeval.metrics import classification_report

In [19]:
import numpy as np
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    
    result = classification_report(true_labels, true_predictions, output_dict = True)

    print(result)
    
    return result['ORG']

In [23]:
args = TrainingArguments(
    save_folder_name,
    save_strategy='epoch',
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epoch,
    weight_decay = weight_decay,
    metric_for_best_model = 'f1-score',
    save_total_limit=5,
    load_best_model_at_end = True
)

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("Training...")
trainer.train()
print("Evaluating...")
trainer.evaluate()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 13856
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3464
  Number of trainable parameters = 108493059


Training...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1-score,Support
1,0.015100,0.092740,0.710068,0.803770,0.754019,2334
2,0.008300,0.104243,0.727958,0.827763,0.774659,2334
3,0.005300,0.093539,0.765014,0.813196,0.788370,2334
4,0.002800,0.117429,0.763644,0.833333,0.796968,2334


***** Running Evaluation *****
  Num examples = 1227
  Batch size = 16


{'ORG': {'precision': 0.7100681302043906, 'recall': 0.803770351328192, 'f1-score': 0.7540192926045017, 'support': 2334}, 'micro avg': {'precision': 0.7100681302043906, 'recall': 0.803770351328192, 'f1-score': 0.7540192926045017, 'support': 2334}, 'macro avg': {'precision': 0.7100681302043906, 'recall': 0.803770351328192, 'f1-score': 0.7540192926045017, 'support': 2334}, 'weighted avg': {'precision': 0.7100681302043906, 'recall': 0.803770351328192, 'f1-score': 0.7540192926045017, 'support': 2334}}


...

{'ORG': {'precision': 0.7636435021594032, 'recall': 0.8333333333333334, 'f1-score': 0.7969678344601516, 'support': 2334}, 'micro avg': {'precision': 0.7636435021594032, 'recall': 0.8333333333333334, 'f1-score': 0.7969678344601516, 'support': 2334}, 'macro avg': {'precision': 0.7636435021594032, 'recall': 0.8333333333333334, 'f1-score': 0.7969678344601516, 'support': 2334}, 'weighted avg': {'precision': 0.7636435021594032, 'recall': 0.8333333333333334, 'f1-score': 0.7969678344601516, 'support': 2334}}


Saving model checkpoint to bert_alldata/checkpoint-3464
Configuration saved in bert_alldata/checkpoint-3464/config.json
Model weights saved in bert_alldata/checkpoint-3464/pytorch_model.bin
tokenizer config file saved in bert_alldata/checkpoint-3464/tokenizer_config.json
Special tokens file saved in bert_alldata/checkpoint-3464/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from bert_alldata/checkpoint-3464 (score: 0.7969678344601516).
***** Running Evaluation *****
  Num examples = 1227
  Batch size = 16


Evaluating...


{'ORG': {'precision': 0.7636435021594032, 'recall': 0.8333333333333334, 'f1-score': 0.7969678344601516, 'support': 2334}, 'micro avg': {'precision': 0.7636435021594032, 'recall': 0.8333333333333334, 'f1-score': 0.7969678344601516, 'support': 2334}, 'macro avg': {'precision': 0.7636435021594032, 'recall': 0.8333333333333334, 'f1-score': 0.7969678344601516, 'support': 2334}, 'weighted avg': {'precision': 0.7636435021594032, 'recall': 0.8333333333333334, 'f1-score': 0.7969678344601516, 'support': 2334}}


{'eval_loss': 0.11742940545082092,
 'eval_precision': 0.7636435021594032,
 'eval_recall': 0.8333333333333334,
 'eval_f1-score': 0.7969678344601516,
 'eval_support': 2334,
 'eval_runtime': 12.2795,
 'eval_samples_per_second': 99.923,
 'eval_steps_per_second': 6.271,
 'epoch': 4.0}

In [24]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 1226
  Batch size = 16


{'ORG': {'precision': 0.7787610619469026, 'recall': 0.8620949510173324, 'f1-score': 0.8183118741058656, 'support': 2654}, 'micro avg': {'precision': 0.7787610619469026, 'recall': 0.8620949510173324, 'f1-score': 0.8183118741058656, 'support': 2654}, 'macro avg': {'precision': 0.7787610619469026, 'recall': 0.8620949510173324, 'f1-score': 0.8183118741058656, 'support': 2654}, 'weighted avg': {'precision': 0.7787610619469028, 'recall': 0.8620949510173324, 'f1-score': 0.8183118741058656, 'support': 2654}}


PredictionOutput(predictions=array([[[ 8.813953  , -4.558598  , -4.912768  ],
        [ 8.489658  , -4.4184027 , -4.6914597 ],
        [ 8.322119  , -4.7403607 , -4.462311  ],
        ...,
        [-0.98701245,  4.8849845 , -2.727016  ],
        [ 8.259553  , -4.5092974 , -4.5522194 ],
        [ 8.843801  , -4.8288174 , -5.0708942 ]],

       [[ 8.22143   , -4.059431  , -5.1734395 ],
        [ 8.305553  , -5.307601  , -4.2488775 ],
        [ 0.19298999,  4.021653  , -4.3085737 ],
        ...,
        [-1.1066588 , -0.89818156,  2.068591  ],
        [ 4.507401  , -1.0333705 , -3.4463887 ],
        [ 1.1045256 ,  0.4105907 , -1.6067594 ]],

       [[ 8.177227  , -4.7960434 , -4.6856365 ],
        [ 8.926936  , -4.963509  , -5.0004377 ],
        [ 8.717618  , -5.057914  , -4.8614683 ],
        ...,
        [ 8.734864  , -5.0360694 , -4.9176598 ],
        [ 8.396253  , -4.6824207 , -4.722383  ],
        [ 2.0196693 , -2.030562  , -1.6602373 ]],

       ...,

       [[ 8.742051  , -4.570662